# Imports and bronze table reading

In [0]:
import pyspark.sql.functions as func
from pyspark.sql.types import StringType
from pyspark.sql.functions import col, trim

In [0]:
df = spark.table("workspace.bronze.erp_cust_az12")
#df.display()

# Transformations

### Trimming

In [0]:
for field in df.schema.fields:
    if isinstance(field.dataType, StringType):
        df = df.withColumn(field.name, trim(col(field.name)))

### Customer ID Cleanup

In [0]:
df = df.withColumn(
    "cid",
    func.when(col("cid").startswith("NAS"),
            func.substring(col("cid"), 4, func.length(col("cid"))))
     .otherwise(col("cid"))
)

### Birthdate Validation

In [0]:
df = df.withColumn(
    "bdate",
    func.when(col("bdate") > func.current_date(), None)
     .otherwise(col("bdate"))
)

### Gender Normalization

In [0]:
df = df.withColumn(
    "gen",
    func.when(func.upper(col("gen")).isin("F", "FEMALE"), "Female")
     .when(func.upper(col("gen")).isin("M", "MALE"), "Male")
     .otherwise("n/a")
)

### Renaming Colums

In [0]:
RENAME_MAP = {
    "cid": "customer_number",
    "bdate": "birth_date",
    "gen": "gender"
}
for old_name, new_name in RENAME_MAP.items():
    df = df.withColumnRenamed(old_name, new_name)

### Sanity checks

In [0]:
df.limit(10).display()

# Writing Silver Table

In [0]:
df.write.mode("overwrite").format("delta").saveAsTable("workspace.silver.erp_customers")

### Sanity checks of silver table

In [0]:
%sql
SELECT * FROM workspace.silver.erp_customers LIMIT 10